# Cloudbutton Geospatial: Water Consumption Workflow

---

In [3]:
import sys
sys.path.append('../')

&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "conn=None">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.history.HistoryManager object at 0x7fb5aa8a7c70>, <sqlite3.Connection object at 0x7fb5aaa8d8a0>)


In [4]:
from collections import defaultdict
from cloudbutton_geospatial.io_utils.plot import plot_results
from cloudbutton_geospatial.utils.notebook import date_picker
from rasterio.windows import Window
from scipy.spatial import distance_matrix
from shapely.geometry import Point, MultiPoint, box
from pprint import pprint
import functools
import collections
import datetime
import os
import shutil
import math
import numpy as np
import pandas as pd
import lithops
import requests
import rasterio
import json
import random
import re
import tempfile
import concurrent.futures
from IPython.display import Image
import matplotlib.pyplot as plt
from lithops.storage import Storage
from lithops.storage.utils import StorageNoSuchKeyError

&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "conn=None">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.history.HistoryManager object at 0x7fb5aa8a7c70>, <sqlite3.Connection object at 0x7fb5aaa8d8a0>)


## Workflow parameters

Area outside the processed tile that we want to consider for taking SIAM stations into account:

In [5]:
AREA_OF_INFLUENCE = 16000

&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "conn=None">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.history.HistoryManager object at 0x7fb5aa8a7c70>, <sqlite3.Connection object at 0x7fb5aaa8d8a0>)


Lithops Variables:

In [6]:
DATA_BUCKET = 'cloudbutton-geospatial-wc'
COMPUTE_BACKEND = 'aws_lambda'
STORAGE_BACKEND = 'aws_s3'
STORAGE_PREFIX = 's3://'
RUNTIME = 'wc:03'
RUNTIME_MEMORY = 2048

&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "conn=None">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.history.HistoryManager object at 0x7fb5aa8a7c70>, <sqlite3.Connection object at 0x7fb5aaa8d8a0>)


In [7]:
DTM_PREFIX = 'DTMs/'
DTM_ASC_PREFIX = 'DTMs/asc/'
DTM_GEOTIFF_PREFIX = 'DTMs/geotiff/'

&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "conn=None">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.history.HistoryManager object at 0x7fb5aa8a7c70>, <sqlite3.Connection object at 0x7fb5aaa8d8a0>)


Split tile into square chunks (number of tiles = SPLITS^2):

In [8]:
SPLITS = 3

&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "conn=None">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.history.HistoryManager object at 0x7fb5aa8a7c70>, <sqlite3.Connection object at 0x7fb5aaa8d8a0>)


Correlation coefficient between elevation and temperature:

In [9]:
r = -0.0056

&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "conn=None">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.history.HistoryManager object at 0x7fb5aa8a7c70>, <sqlite3.Connection object at 0x7fb5aaa8d8a0>)


Elevation to interpolate temperature:

In [10]:
zdet = 2000

&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "conn=None">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.history.HistoryManager object at 0x7fb5aa8a7c70>, <sqlite3.Connection object at 0x7fb5aaa8d8a0>)


Day of year to calculate solar irradiation:

In [11]:
date = date_picker(default=datetime.date(2022, 5, 15))

&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "conn=None">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.history.HistoryManager object at 0x7fb5aa8a7c70>, <sqlite3.Connection object at 0x7fb5aaa8d8a0>)
&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "obj">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.formatters.IPythonDisplayFormatter object at 0x7fb56fcbdf10>, DatePicker(value=datetime.date(2022, 5, 15), description='Pick a Date', step=1))
&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "obj">, <Parameter "include=None">, <Parameter "exclude=None">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.formatters.MimeBundleFormatter object at 0x7fb598083310>, DatePicker(value=datetime.date(2022, 5, 15), description='Pick a Date', step=1))
&&&&&&&&&&&&&&&&&&&&&& param include=None
&&&&&&&&&&&&&&&&&&&&&& param.kind POSITIONAL_OR_KEYWORD
&&&&&&&&&&&&&&&&&&& self.parameters.values()

DatePicker(value=datetime.date(2022, 5, 15), description='Pick a Date', step=1)

In [12]:
DAY_OF_YEAR = date.value.timetuple().tm_yday
DAY_OF_YEAR

&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "obj">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.formatters.IPythonDisplayFormatter object at 0x7fb56fcbdf10>, 135)
&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "obj">, <Parameter "include=None">, <Parameter "exclude=None">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.formatters.MimeBundleFormatter object at 0x7fb598083310>, 135)
&&&&&&&&&&&&&&&&&&&&&& param include=None
&&&&&&&&&&&&&&&&&&&&&& param.kind POSITIONAL_OR_KEYWORD
&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "obj">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.formatters.PlainTextFormatter object at 0x7fb5a8058e50>, 135)
&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "obj">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.formatters.HTMLFormatter object at 0x7fb5a8068520>, 135)
&&&&&&&&&&&&&&&&&&& self.parameters.value

135

Initialize Lithops Storage and Function Executor:

In [13]:
storage = lithops.storage.Storage(backend=STORAGE_BACKEND)

&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "conn=None">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.history.HistoryManager object at 0x7fb5aa8a7c70>, <sqlite3.Connection object at 0x7fb5aaa8d8a0>)


In [14]:
fexec = lithops.FunctionExecutor(backend=COMPUTE_BACKEND, storage=STORAGE_BACKEND, runtime=RUNTIME, runtime_memory=RUNTIME_MEMORY)

2023-09-12 00:20:28,815 [INFO] config.py:141 -- Lithops v2.9.1.dev0 - Python3.8
2023-09-12 00:20:28,828 [INFO] aws_s3.py:68 -- S3 client created - Region: us-west-2


&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "conn=None">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.history.HistoryManager object at 0x7fb5aa8a7c70>, <sqlite3.Connection object at 0x7fb5aaa8d8a0>)


2023-09-12 00:20:30,918 [INFO] aws_lambda.py:94 -- AWS Lambda client created - Region: us-west-2


## Data preparation

### SIAM data

In [15]:
siam_data_key = 'siam_data.csv'
try:
    siam_data_head = storage.head_object(bucket=DATA_BUCKET, key=siam_data_key)
    print(f'SIAM meteo data already in storage: {siam_data_head}')
except StorageNoSuchKeyError:
    print('Uploading SIAM meteo data to Object Storage...')
    with open(siam_data_key, 'rb') as f:
        storage.put_object(bucket=DATA_BUCKET, key=siam_data_key, body=f)

&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "conn=None">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.history.HistoryManager object at 0x7fb5aa8a7c70>, <sqlite3.Connection object at 0x7fb5aaa8d8a0>)
SIAM meteo data already in storage: {'x-amz-id-2': 'XxAKVGIv6IRjb9PF6TdFL63mMzd2AH41tTlxoWv3XO70DeCQlhT+mSOPrLNqatWLtcmPeQBRfu8=', 'x-amz-request-id': 'D8VY2B3VYC282H28', 'date': 'Tue, 12 Sep 2023 07:20:51 GMT', 'last-modified': 'Tue, 12 Sep 2023 03:34:51 GMT', 'etag': '"8a1fd5da76b1123e66cc0155e6c8f5f7"', 'x-amz-server-side-encryption': 'AES256', 'accept-ranges': 'bytes', 'content-type': 'binary/octet-stream', 'server': 'AmazonS3', 'content-length': '3850'}


### Shapefile

In [16]:
shapefile_key = 'shapefile_murcia.zip'
try:
    shapefile_head = storage.head_object(bucket=DATA_BUCKET, key=shapefile_key)
    print(f'Shapefile already in storage: {siam_data_head}')
except StorageNoSuchKeyError:
    print('Uploading shapefile to Object Storage...')
    with open(shapefile_key, 'rb') as f:
        storage.put_object(bucket=DATA_BUCKET, key=shapefile_key, body=f)

&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "conn=None">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.history.HistoryManager object at 0x7fb5aa8a7c70>, <sqlite3.Connection object at 0x7fb5aaa8d8a0>)
Shapefile already in storage: {'x-amz-id-2': 'XxAKVGIv6IRjb9PF6TdFL63mMzd2AH41tTlxoWv3XO70DeCQlhT+mSOPrLNqatWLtcmPeQBRfu8=', 'x-amz-request-id': 'D8VY2B3VYC282H28', 'date': 'Tue, 12 Sep 2023 07:20:51 GMT', 'last-modified': 'Tue, 12 Sep 2023 03:34:51 GMT', 'etag': '"8a1fd5da76b1123e66cc0155e6c8f5f7"', 'x-amz-server-side-encryption': 'AES256', 'accept-ranges': 'bytes', 'content-type': 'binary/octet-stream', 'server': 'AmazonS3', 'content-length': '3850'}


### Digital Terrain Models

Download DTM files for free from http://centrodedescargas.cnig.es/CentroDescargas/buscadorCatalogo.do?codFamilia=MDT05# and put them in `input_DTMs` folder.

In [17]:
dtm_asc_keys = storage.list_keys(bucket=DATA_BUCKET, prefix=DTM_ASC_PREFIX)
dtm_asc_keys

&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "conn=None">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.history.HistoryManager object at 0x7fb5aa8a7c70>, <sqlite3.Connection object at 0x7fb5aaa8d8a0>)
&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "obj">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.formatters.IPythonDisplayFormatter object at 0x7fb56fcbdf10>, ['DTMs/asc/PNOA_MDT05_ETRS89_HU30_0818_LID.asc'])
&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "obj">, <Parameter "include=None">, <Parameter "exclude=None">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.formatters.MimeBundleFormatter object at 0x7fb598083310>, ['DTMs/asc/PNOA_MDT05_ETRS89_HU30_0818_LID.asc'])
&&&&&&&&&&&&&&&&&&&&&& param include=None
&&&&&&&&&&&&&&&&&&&&&& param.kind POSITIONAL_OR_KEYWORD
&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "obj">])
&&&&&&&

['DTMs/asc/PNOA_MDT05_ETRS89_HU30_0818_LID.asc']

Find downloaded MDTs:

In [18]:
local_dtm_input = 'input_DTMs'
print("%%%%%%%%%%%%",os.listdir(local_dtm_input))
local_dtms = [os.path.join(local_dtm_input, dtm) for dtm in os.listdir(local_dtm_input) if dtm.endswith('.asc')]
print("*****", local_dtms)
def upload_file(file_path):
    key = os.path.join(DTM_ASC_PREFIX, os.path.basename(file_path))
    if key in dtm_asc_keys:
        print(f'Tile {key} already in storage')
        return key
    with open(file_path, 'rb') as f:
        print(f'Uploading {key}...')
        storage.put_object(bucket=DATA_BUCKET, key=key, body=f)
    return key

with concurrent.futures.ThreadPoolExecutor(max_workers=1) as pool:
    result = list(pool.map(upload_file, local_dtms))
    # list(result)

%%%%%%%%%%%% ['.gitkeep', 'PNOA_MDT05_ETRS89_HU30_0818_LID.tif', 'PNOA_MDT05_ETRS89_HU30_0818_LID.asc']
&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "conn=None">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.history.HistoryManager object at 0x7fb5aa8a7c70>, <sqlite3.Connection object at 0x7fb5aaa8d8a0>)
***** ['input_DTMs/PNOA_MDT05_ETRS89_HU30_0818_LID.asc']
Tile DTMs/asc/PNOA_MDT05_ETRS89_HU30_0818_LID.asc already in storage


Convert ASCII raster files to Cloud Optimized GeoTIFF:

In [25]:
print(os.path.join(STORAGE_PREFIX, DATA_BUCKET, DTM_ASC_PREFIX))

s3://cloudbutton-geospatial-wc/DTMs/asc/
&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "conn=None">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.history.HistoryManager object at 0x7fb5aa8a7c70>, <sqlite3.Connection object at 0x7fb5aaa8d8a0>)


In [22]:
def asc_to_geotiff(obj, storage):
    print("$$$$$$$$$$$$$$$ obj", obj)
    print("$$$$$$$$$$$$$$$ obj.key", obj.key)
    print("$$$$$$$$$$$$$$$ obj.data_stream", obj.data_stream)
    asc_file_name = os.path.basename(obj.key)
    tile_id, _ = os.path.splitext(asc_file_name)
    out_path = os.path.join(tempfile.gettempdir(), tile_id + '.tiff')
    out_key = os.path.join(DTM_GEOTIFF_PREFIX, tile_id + '.tiff')
    
    try:
        out_obj = storage.head_object(bucket=DATA_BUCKET, key=out_key)
    except StorageNoSuchKeyError:
        out_obj = None
    
    if out_obj:
        print(f'GeoTIFF {tile_id} already exists, skipping...')
        return out_key
    
    print(f'Converting {tile_id} to GeoTIFF...')
    with rasterio.open(obj.data_stream, 'r') as src:
        profile = src.profile
        # Cloud optimized GeoTiff parameters
        profile.update(driver='GTiff')
        profile.update(blockxsize=256)
        profile.update(blockysize=256)
        profile.update(tiled=True)
        profile.update(compress='deflate')
        profile.update(interleave='band')
        with rasterio.open(out_path, 'w', **profile) as dest:
            dest.write(src.read())
    
    with open(out_path, 'rb') as f:
        storage.put_object(bucket=DATA_BUCKET, key=out_key, body=f)
    
    return out_key

&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "conn=None">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.history.HistoryManager object at 0x7fb5aa8a7c70>, <sqlite3.Connection object at 0x7fb5aaa8d8a0>)


In [23]:
fs_cog = fexec.map(asc_to_geotiff, os.path.join(STORAGE_PREFIX, DATA_BUCKET, DTM_ASC_PREFIX))

2023-09-12 00:49:19,790 [INFO] invokers.py:108 -- ExecutorID eece82-0 | JobID M001 - Selected Runtime: wc:03 - 2048MB


&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "conn=None">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.history.HistoryManager object at 0x7fb5aa8a7c70>, <sqlite3.Connection object at 0x7fb5aaa8d8a0>)
*******************iterdata s3://cloudbutton-geospatial-wc/DTMs/asc/
*******************extra_args None
*******************data ['s3://cloudbutton-geospatial-wc/DTMs/asc/']
*******************func <function asc_to_geotiff at 0x7fb56b2dedc0>
*******************new_parameters [<Parameter "obj">]
&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "obj">])
&&&&&&&&&&&&&&&&&&& args ('s3://cloudbutton-geospatial-wc/DTMs/asc/',)


2023-09-12 00:49:21,923 [INFO] invokers.py:172 -- ExecutorID eece82-0 | JobID M001 - Starting function invocation: asc_to_geotiff() - Total: 1 activations
2023-09-12 00:49:21,930 [INFO] invokers.py:208 -- ExecutorID eece82-0 | JobID M001 - View execution logs at /tmp/lithops-lithops/logs/eece82-0-M001.log


In [24]:
_, _ = fexec.wait(fs=fs_cog)

2023-09-12 00:50:13,050 [INFO] wait.py:98 -- ExecutorID eece82-0 - Waiting for 100% of 1 function activations to complete


&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "conn=None">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.history.HistoryManager object at 0x7fb5aa8a7c70>, <sqlite3.Connection object at 0x7fb5aaa8d8a0>)
&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "obj">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.formatters.IPythonDisplayFormatter object at 0x7fb56fcbdf10>,     0%|          | 0/1  )
&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "obj">, <Parameter "include=None">, <Parameter "exclude=None">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.formatters.MimeBundleFormatter object at 0x7fb598083310>,     0%|          | 0/1  )
&&&&&&&&&&&&&&&&&&&&&& param include=None
&&&&&&&&&&&&&&&&&&&&&& param.kind POSITIONAL_OR_KEYWORD
&&&&&&&&&&&&&&&&&&& self.parameters.values() odict_values([<Parameter "self">, <Parameter "obj">])
&&&&&&&&&&&&&&&&&&& args (<IPython.core.formatters.HTML

    0%|          | 0/1  

2023-09-12 00:50:13,065 [INFO] executors.py:593 -- ExecutorID eece82-0 - Cleaning temporary data


RasterioIOError: Read or write failed. /vsimem/985511eb-add8-4eec-83aa-eda8d39eb8d8/985511eb-add8-4eec-83aa-eda8d39eb8d8.tif, band 1: IReadBlock failed at X offset 0, Y offset 2074: /vsimem/985511eb-add8-4eec-83aa-eda8d39eb8d8/985511eb-add8-4eec-83aa-eda8d39eb8d8.tif, band 1: File short, can't read line 2074.

In [ ]:
dtm_geotiff_keys = storage.list_keys(bucket=DATA_BUCKET, prefix=DTM_GEOTIFF_PREFIX)
# dtm_geotiff_keys

Get bounds and tile ID for every tile:

In [ ]:
def get_tile_meta(obj):
    tile_id = os.path.splitext(os.path.basename(obj.key))[0]
    with rasterio.open(obj.data_stream, 'r') as src:
        x1, y1 = src.profile['transform'] * (0, 0)
        x2, y2 = src.profile['transform'] * (src.profile['width'], src.profile['height'])
    return tile_id, (x1, y1), (x2, y2)

In [ ]:
fs_meta = fexec.map(get_tile_meta, os.path.join(STORAGE_PREFIX, DATA_BUCKET, DTM_GEOTIFF_PREFIX))

In [ ]:
tiles_meta = fexec.get_result(fs=fs_meta)

In [ ]:
# tiles_meta

In [ ]:
import ipyleaflet
import ipywidgets
import utm

m = ipyleaflet.Map(center=(38.082906, -1.330466), zoom=9.5)

for tile_id, bound1, bound2 in tiles_meta:
    x1, y1 = bound1
    x2, y2 = bound2
    xc, yc = (x1 + x2) / 2, y1
    
    wgs_bounds_1 = utm.to_latlon(x1, y1, 30, 'S')
    wgs_bounds_2 = utm.to_latlon(x2, y2, 30, 'S')
    # wgs_bounds_c = utm.to_latlon(xc, yc, 30, 'S')

    rectangle = ipyleaflet.Rectangle(bounds=(wgs_bounds_1, wgs_bounds_2))
    m.add_layer(rectangle)    

m.layout.height="750px"

m

## Raster Data Interpolation

Split data tiles in subtiles for increased parallelism:

In [ ]:
def data_chunker(obj, n_splits, block_x, block_y, storage):
    tile_key = os.path.basename(obj.key)
    tile_id, _ = os.path.splitext(tile_key)
    
    with rasterio.open(obj.data_stream) as src:
        transform = src.transform
        
        # Compute working window
        step_w = src.width / n_splits
        step_h = src.height / n_splits
        
        offset_h = round(step_h * block_x)
        offset_w = round(step_w * block_y)
        
        profile = src.profile
        
        width = math.ceil(step_w * (block_y + 1) - offset_w)
        height = math.ceil(step_h * (block_x + 1) - offset_h)
        
        profile.update(width=width)
        profile.update(height=height)
        
        window = Window(offset_w, offset_h, width, height)
        
        chunk_file = os.path.join(tempfile.gettempdir(), tile_id + str(block_x) + '_' + str(block_y) + '.tif')
        with rasterio.open(chunk_file, 'w', **profile) as dest:
            dest.write(src.read(window=window))
    
    with open(chunk_file, 'rb') as f:
        co = storage.put_cloudobject(body=f, bucket=DATA_BUCKET)
    
    return (tile_key, block_x, block_y, co)

In [ ]:
iterdata = [(os.path.join(STORAGE_PREFIX, DATA_BUCKET, tile), SPLITS, i, j)
            for i in range(SPLITS) for j in range(SPLITS) for tile in dtm_geotiff_keys]
print(f'Chunking {len(dtm_geotiff_keys)} tiles in {SPLITS * SPLITS} chunks each using {len(iterdata)} functions')
# iterdata

In [ ]:
chunker_fs = fexec.map(data_chunker, iterdata)

In [ ]:
chunks = fexec.get_result(fs=chunker_fs)

In [ ]:
# chunks

Compute solar irradiation for a given day of year using GRASS libraries:

In [ ]:
def compute_solar_irradiation(inputFile, outputFile, crs='32630'):
    # Define grass working set
    GRASS_GISDB = 'grassdata'
    GRASS_LOCATION = 'GEOPROCESSING'
    GRASS_MAPSET = 'PERMANENT'
    GRASS_ELEVATIONS_FILENAME = 'ELEVATIONS'

    os.environ['GRASSBIN'] = 'grass76'
    from grass_session import Session
    import grass.script as gscript
    from grass.pygrass.modules.shortcuts import general
    from grass.pygrass.modules.shortcuts import raster

    os.environ.update(dict(GRASS_COMPRESS_NULLS='1'))
    
    # Clean previously processed data
    if os.path.isdir(GRASS_GISDB):
        shutil.rmtree(GRASS_GISDB)
    
    with Session(gisdb=GRASS_GISDB, location=GRASS_LOCATION, mapset=GRASS_MAPSET, create_opts='EPSG:32630') as ses:
        # Set project projection to match elevation raster projection
        general.proj(epsg=crs, flags='c') 
    
        # Load raster file into working directory
        raster.import_(input=inputFile, output=GRASS_ELEVATIONS_FILENAME, flags='o')    
        
        # Set project region to match raster region
        general.region(raster=GRASS_ELEVATIONS_FILENAME, flags='s')    
        # Calculate solar irradiation
        gscript.run_command('r.slope.aspect', elevation=GRASS_ELEVATIONS_FILENAME,
                            slope='slope', aspect='aspect')
        gscript.run_command('r.sun', elevation=GRASS_ELEVATIONS_FILENAME,
                            slope='slope', aspect='aspect', beam_rad='beam',
                            step=1, day=DAY_OF_YEAR)
        
        # Get extraterrestrial irradiation from history metadata
        regex = re.compile(r'\d+\.\d+')
        output = gscript.read_command("r.info", flags="h", map=["beam"])
        splits = str(output).split('\n')
        line = next(filter(lambda line: 'Extraterrestrial' in line, splits))
        extraterrestrial_irradiance = float(regex.search(line)[0])
        
        # Export generated results into a GeoTiff file
        if os.path.isfile(outputFile):
            os.remove(outputFile)

        raster.out_gdal(input='beam', output=outputFile)
        
        return extraterrestrial_irradiance

Get stations contained in the area of interest:

In [ ]:
def filter_stations(bounds, stations):
    total_points = MultiPoint([Point(x,y) for x, y in stations[['X', 'Y']].to_numpy()])
    intersection = bounds.buffer(AREA_OF_INFLUENCE).intersection(total_points)
    
    return stations[[ intersection.contains(point) for point in total_points]]

Inverse Distance Weighting interpolation:

In [ ]:
def compute_basic_interpolation(shape, stations, field_value, offset = (0,0)):
    station_pixels = [[pixel[0], pixel[1]] for pixel in stations['pixel'].to_numpy()]
    
    # Get an array where each position represents pixel coordinates
    tile_pixels = np.indices(shape).transpose(1,2,0).reshape(shape[0]*shape[1], 2) + offset
    dist = distance_matrix(station_pixels, tile_pixels)
    weights = np.where(dist == 0, np.finfo('float32').max, 1.0 / dist )
    weights /=  weights.sum(axis=0)
    
    return np.dot(weights.T, stations[field_value].to_numpy()).reshape(shape).astype('float32')

Interpolate temperatures from a subset of the tile:

In [ ]:
def radiation_interpolation(tile_key, block_x, block_y, chunk_cloudobject, storage):
    tile_id, _ = os.path.splitext(tile_key)
    print(tile_id)
    
    # Write tile chunk to file
    chunk_file = os.path.join(tempfile.gettempdir(), tile_id + str(block_x) + '_' + str(block_y) + '.tif')
    print(chunk_file)
    with open(chunk_file, 'wb') as f:
        body = storage.get_cloudobject(chunk_cloudobject)
        f.write(body)
    
    with rasterio.open(chunk_file, 'r') as chunk_src:
        profile = chunk_src.profile
        
    extr_chunk_file = os.path.join(tempfile.gettempdir(), tile_id + '_extr_' + str(block_x) + '_' + str(block_y) + '.tif')
    rad_chunk_file = os.path.join(tempfile.gettempdir(), tile_id + '_rad_' + str(block_x) + '_' + str(block_y) + '.tif')
    
    # Compute solar irradiation from inputFile, creates radiation raster at outputFile
    extraterrestrial_irradiation = compute_solar_irradiation(inputFile=chunk_file, outputFile=rad_chunk_file)
        
    # Create and store a raster with extraterrestrial irradiation
    with rasterio.open(extr_chunk_file, 'w', **profile) as dest:
        data = np.full((profile['height'], profile['width']), extraterrestrial_irradiation, dtype='float32')
        dest.write(data, 1)
    
    with open(extr_chunk_file, 'rb') as f:
        extr_co = storage.put_cloudobject(body=f, bucket=DATA_BUCKET)
    
    with open(rad_chunk_file, 'rb') as f:
        rad_co = storage.put_cloudobject(body=f, bucket=DATA_BUCKET)
    
    return [(tile_key, 'extr', block_x, block_y, extr_co), (tile_key, 'rad', block_x, block_y, rad_co)]

In [ ]:
def map_interpolation(tile_key, block_x, block_y, chunk_cloudobject, data_field, storage):
    tile_id, _ = os.path.splitext(tile_key)
    
    # Get SIAM meteo data
    siam_stream = storage.get_object(DATA_BUCKET, siam_data_key, stream=True)
    siam_data = pd.read_csv(siam_stream)
    
    chunk = storage.get_cloudobject(chunk_cloudobject, stream=True)
    
    with rasterio.open(chunk) as chunk_src:
        transform = chunk_src.transform
        profile = chunk_src.profile
        
        bounding_rect = box(chunk_src.bounds.left, chunk_src.bounds.top, chunk_src.bounds.right, chunk_src.bounds.bottom)
        filtered = pd.DataFrame(filter_stations(bounding_rect, siam_data))
        
        if filtered.shape[0] == 0:
            return [(tile_key, data_field, block_x, block_y, None)]
        
        filtered['pixel'] = filtered.apply(lambda station: rasterio.transform.rowcol(transform, station['X'], station['Y']), axis=1)
                
        # Interpolate variables from meteo station data, generate raster with result
        dest_chunk_file = os.path.join(tempfile.gettempdir(), tile_id + '_' + data_field + '_' + str(block_x) + '_' + str(block_y) + '.tif')
        
        with rasterio.open(dest_chunk_file, 'w', **profile) as chunk_dest:
            if data_field == 'temp':
                elevations = chunk_src.read(1)  # Get elevations content
                interpolation = compute_basic_interpolation(elevations.shape, filtered, 'tdet', (0, 0))
                interpolation += r * (elevations - zdet)
                chunk_dest.write(np.where(elevations == chunk_src.nodata, np.nan, interpolation), 1)
            elif data_field == 'humi':
                interpolation = compute_basic_interpolation((profile['height'], profile['width']), filtered, 'hr', (0, 0))
                chunk_dest.write(interpolation, 1)
            elif data_field == 'wind':
                interpolation = compute_basic_interpolation((profile['height'], profile['width']), filtered, 'v', (0, 0))
                chunk_dest.write(interpolation, 1)
            else:
                raise Exception(f'Unknown data field "{data_field}"')

    # Upload results to storage as Cloudobject
    with open(dest_chunk_file, 'rb') as f:
        co = storage.put_cloudobject(body=f, bucket=DATA_BUCKET)
    
    return [(tile_key, data_field, block_x, block_y, co)]

Lithops serverless computation:

In [ ]:
fs_rad = fexec.map(radiation_interpolation, chunks, runtime_memory=2048)
fs_temp = fexec.map(map_interpolation, chunks, extra_args=('temp', ), runtime_memory=2048)
fs_humi = fexec.map(map_interpolation, chunks, extra_args=('humi', ), runtime_memory=2048)
fs_wind = fexec.map(map_interpolation, chunks, extra_args=('wind', ), runtime_memory=2048)

In [ ]:
res_rad = fexec.get_result(fs=fs_rad)
res_temp = fexec.get_result(fs=fs_temp)
res_humi = fexec.get_result(fs=fs_humi)
res_wind = fexec.get_result(fs=fs_wind)

In [ ]:
res_flatten = []
for l in [res_rad, res_temp, res_humi, res_wind]:
    for elem in l:
        for sub_elem in elem:
            res_flatten.append(sub_elem)

In [ ]:
# res_flatten

In [ ]:
grouped_chunks = collections.defaultdict(list)

for chunk_result in res_flatten:
    tile_key, data_field, block_x, block_y, co = chunk_result
    grouped_chunks[(tile_key, data_field)].append((block_x, block_y, co))

In [ ]:
# grouped_chunks

Join split subsets into a tile:

In [ ]:
def merge_blocks(tile_data, chunks, storage):
    from rasterio.windows import Window
    
    tile_key, data_field = tile_data
    
    cobjs = [tup[2] for tup in chunks]
    if None in cobjs:
        return None
    
    # Get width and height from original tile
    source_tile_key = os.path.join(DTM_GEOTIFF_PREFIX, tile_key)
    with rasterio.open(storage.get_object(bucket=DATA_BUCKET, key=source_tile_key, stream=True)) as source_tile:
        height = source_tile.profile['height']
        width = source_tile.profile['width']
        
    # Open first object to obtain profile metadata
    with rasterio.open(storage.get_cloudobject(chunks[0][2], stream=True)) as chunk_src:
        profile = chunk_src.profile
        profile.update(width=width)
        profile.update(height=height)

    # Iterate each object and print its block into the destination file
    merged_file = os.path.join(tempfile.gettempdir(), data_field + '_' + tile_key)
    with rasterio.open(merged_file, 'w', **profile) as dest: 
        for chunk in chunks:
            j, i, co = chunk
            with rasterio.open(storage.get_cloudobject(co, stream=True)) as src:
                step_w = math.floor(width / SPLITS)
                step_h = math.floor(height / SPLITS)
                curr_window = Window(round(step_w * i), round(step_h * j), src.width, src.height)
                content = src.read(1)
                dest.write(content, 1, window=curr_window)
    
    output_key = os.path.join(DTM_PREFIX, data_field, tile_key)
    with open(merged_file, 'rb') as out_file:
        storage.put_object(bucket=DATA_BUCKET, key=output_key, body=out_file)  
    
    return output_key

Combine previous split subsets:

In [ ]:
iterdata = []
for (tile_id, data_field), chunks in grouped_chunks.items():
    iterdata.append(((tile_id, data_field), chunks))

In [ ]:
fs_merged = fexec.map(merge_blocks, iterdata, runtime_memory=2048)

In [ ]:
tiles_merged = fexec.get_result(fs=fs_merged)

In [ ]:
tile_keys_merged = set([os.path.basename(t) for t in tiles_merged])

In [ ]:
# tile_keys_merged

## Computation of potential evaporation

In [ ]:
def compute_crop_evapotranspiration(temperatures,
                                    humidities,
                                    wind_speeds,
                                    external_radiations,
                                    global_radiations,
                                    KCs):
    gamma = 0.665*101.3/1000
    eSat = 0.6108 * np.exp((17.27*temperatures)/(temperatures+237.3))
    delta = 4098 * eSat / np.power((temperatures + 237.3),2)
    eA = np.where(humidities < 0, 0, eSat * humidities / 100)     # Avoid sqrt of a negative number
    T4 = 4.903 * np.power((273.3 + temperatures),4)/1000000000
    rSrS0 = global_radiations/(external_radiations * 0.75)
    rN = 0.8* global_radiations-T4*(0.34-0.14*np.sqrt(eA))*((1.35*rSrS0)-0.35)
    den = delta + gamma *(1 + 0.34* wind_speeds)
    tRad = 0.408 * delta * rN / den
    tAdv = gamma * (900/(temperatures+273))*wind_speeds * (eSat - eA)/den
    return ((tRad + tAdv) * 7 * KCs).astype('float32')

In [ ]:
vineyard = ['VI', 'VO', 'VF', 'FV', 'CV' ]
olive_grove = ['OV', 'VO', 'OF', 'FL', 'OC']
fruit = ['FY', 'VF', 'OF', 'FF', 'CF']
nuts = ['FS', 'FV', 'FL', 'FF', 'CS' ]
citrus = ['CI', 'CV', 'OC', 'CF', 'CS' ]

def get_kc(feature):
    # TODO: Get more precise values of Kc
    print(feature['properties'].keys())
    # sigpac_use = feature['properties']['uso_sigpac']
    sigpac_use = 'FF'
    if sigpac_use in vineyard:
        # Grapes for wine - 0.3, 0.7, 0.45
        return 0.7  
    if sigpac_use in olive_grove:
        # Olive grove - ini: 0.65, med: 0.7, end: 0.7
        return 0.7 
    if sigpac_use in fruit:
        # Apples, Cherries, Pears - 0.45, 0.95, 0.7
        return 0.95
    if sigpac_use in nuts:
        # Almonds - 0.4, 0.9, 0.65
        return 0.9
    if sigpac_use in citrus:
        # Citrus, without ground coverage - 0.7, 0.65, 0.7
        return 0.65
    
    return None

In [ ]:
def get_geometry_window(src, geom_bounds):
    left, bottom, right, top = geom_bounds
    src_left, src_bottom, src_right, src_top = src.bounds
    window = src.window(max(left,src_left), max(bottom,src_bottom), min(right,src_right), min(top,src_top))
    window_floored = window.round_offsets(op='floor', pixel_precision=3)
    w = math.ceil(window.width + window.col_off - window_floored.col_off)
    h = math.ceil(window.height + window.row_off - window_floored.row_off)
    return Window(window_floored.col_off, window_floored.row_off, w, h)     

In [ ]:
def compute_evapotranspiration_by_shape(tem, hum, win, rad, extrad, dst):
    
    import fiona
    from shapely.geometry import shape, box
    from rasterio import features
    
    non_arable_land = ['AG', 'CA', 'ED', 'FO', 'IM', 'PA', 'PR', 'ZU', 'ZV']
    
    with fiona.open('zip://shape.zip') as shape_src:
        for feature in shape_src.filter(bbox=tem.bounds):
            KC = get_kc(feature) 
            if KC is not None:   
                geom = shape(feature['geometry'])  
                window = get_geometry_window(tem, geom.bounds)              
                win_transform = rasterio.windows.transform(window, tem.transform)
                # Convert shape to raster matrix
                image = features.rasterize([geom],
                                           out_shape=(window.height, window.width),
                                           transform = win_transform,
                                           fill = 0,
                                           default_value = 1).astype('bool')
                # Get values to compute evapotranspiration
                temperatures = tem.read(1, window=window)
                humidities = hum.read(1, window=window)
                wind_speeds = win.read(1, window=window)
                # Convert from W to MJ (0.0036)
                global_radiations = rad.read(1, window=window) * 0.0036
                external_radiations = extrad.read(1, window=window) * 0.0036
                KCs = np.full(temperatures.shape, KC)
                # TODO: compute external radiation
                #external_radiations = np.full(temperatures.shape, 14)
                # TODO: compute global radiation
                # global_radiations = np.full(temperatures.shape, 10)
                etc = compute_crop_evapotranspiration(
                        temperatures,
                        humidities,
                        wind_speeds,
                        external_radiations,
                        global_radiations,
                        KCs
                )
                etc[temperatures == tem.nodata] = dst.nodata
                etc[np.logical_not(image)] = dst.nodata
                dst.write(etc + dst.read(1, window=window), 1, window=window)

In [ ]:
def compute_global_evapotranspiration(tem, hum, win, rad, extrad, dst):    
    for ji, window in tem.block_windows(1):
        bounds = rasterio.windows.bounds(window, tem.transform)
        temperatures = tem.read(1, window=window)
        humidities = hum.read(1, window=window)
        wind_speeds = win.read(1, window=window)
         # Convert from W to MJ (0.0036)
        global_radiations = rad.read(1, window=window) * 0.0036
        external_radiations = extrad.read(1, window=window) * 0.0036
        # TODO: compute external radiation
        #external_radiations = np.full(temperatures.shape, 14)
        # TODO: compute global radiation
        # global_radiations = np.full(temperatures.shape, 10)
        # TODO: compute KCs
        KCs = np.full(temperatures.shape, 1)
        etc = compute_crop_evapotranspiration(
                temperatures,
                humidities,
                wind_speeds,
                external_radiations,
                global_radiations,
                KCs
        )
        dst.write(np.where(temperatures == tem.nodata, dst.nodata, etc), 1, window=window)

In [ ]:
def combine_calculations(tile_key, storage):
    
    from functools import partial
      
    # Download shapefile
    shapefile = storage.get_object(bucket=DATA_BUCKET, key='shapefile_murcia.zip', stream=True)
    with open('shape.zip', 'wb') as shapf:
        for chunk in iter(partial(shapefile.read, 200 * 1024 * 1024), ''):
            if not chunk:
                break
            shapf.write(chunk)
    try:
        temp = storage.get_object(bucket=DATA_BUCKET, key=os.path.join(DTM_PREFIX, 'temp', tile_key), stream=True)
        humi = storage.get_object(bucket=DATA_BUCKET, key=os.path.join(DTM_PREFIX, 'humi', tile_key), stream=True)
        rad = storage.get_object(bucket=DATA_BUCKET, key=os.path.join(DTM_PREFIX, 'rad', tile_key), stream=True)
        extrad = storage.get_object(bucket=DATA_BUCKET, key=os.path.join(DTM_PREFIX, 'extr', tile_key), stream=True)
        wind = storage.get_object(bucket=DATA_BUCKET, key=os.path.join(DTM_PREFIX, 'wind', tile_key), stream=True)
    except StorageNoSuchKeyError:
        return None
    
    output_file = os.path.join(tempfile.gettempdir(), 'eva' + '_' + tile_key)
    with rasterio.open(temp) as temp_raster:
        with rasterio.open(humi) as humi_raster:
            with rasterio.open(rad) as rad_raster:
                with rasterio.open(extrad) as extrad_raster:
                    with rasterio.open(wind) as wind_raster:
                        profile = temp_raster.profile
                        profile.update(nodata=0)
                        with rasterio.open(output_file, 'w+', **profile) as dst:
#                             compute_global_evapotranspiration(temp_raster, humi_raster, wind_raster,
#                                                               rad_raster, extrad_raster, dst)
                            compute_evapotranspiration_by_shape(temp_raster, humi_raster, wind_raster,
                                                                rad_raster, extrad_raster, dst)
    
    output_key = os.path.join(DTM_PREFIX, 'eva', tile_key)
    with open(output_file, 'rb') as output_f:
        storage.put_object(bucket=DATA_BUCKET, key=output_key, body=output_f)
    return output_key

In [ ]:
fs_eva = fexec.map(combine_calculations, tile_keys_merged, runtime_memory=2048)
res_eva = fexec.get_result(fs=fs_eva)

In [ ]:
res_eva

---

In [ ]:
fexec.plot(dst=fexec.executor_id)

In [ ]:
fexec.clean(clean_cloudobjects=True)

In [ ]:
fexec.job_summary()

In [ ]:
len(dtm_asc_keys)

In [ ]:
input_sz = 0
for input_key in dtm_asc_keys:
    meta = storage.head_object(bucket=DATA_BUCKET, key=input_key)
    input_sz += int(meta['content-length'])

In [ ]:
input_sz

---

## Visualization of results

In [ ]:
tile_key = os.path.basename(tile)
tile_id, _ = os.path.splitext(tile_key)

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots()

with rasterio.open('PNOA_MDT05_ETRS89_HU30_0891_LID.tiff') as src:
    arr = src.read(1, out_shape=(src.height, src.width))
    ax.set_title(tile_id)
    img = ax.imshow(arr, cmap='Greens')
    fig.colorbar(img, shrink=0.5)

fig.set_size_inches(18.5, 10.5)
plt.show()

# obj.seek(0)

---